In [1]:
import pandas as pd 
import numpy as np 

from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('../data/ABT_mgn.csv')

In [3]:
df

,class,active_region,start_time,end_time,TOTUSJH_Max,TOTUSJH_Min,TOTUSJH_Median,TOTUSJH_Mean,TOTUSJH_Std_dev,TOTUSJH_Variance,...,R_VALUE_Mean,R_VALUE_Std_dev,R_VALUE_Variance,R_VALUE_Skewness,R_VALUE_Kurtosis,R_VALUE_avg_abs_derivative_change,R_VALUE_last_value,R_VALUE_average_absolute_change,R_VALUE_quadratic_weighted_moving_average,R_VALUE_linear_weighted_moving_average
0,1,3688,2014-01-31T19:00:00,2014-02-01T06:48:00,2234.206232,1812.025600,1936.642763,1966.858921,111.419294,12414.259087,...,4.172024,0.125432,0.015733,-0.264967,-0.965933,0.004286,4.351332,0.035149,4.199851,4.176571
1,1,3366,2013-11-12T16:12:00,2013-11-13T04:00:00,4350.649277,4118.376480,4226.909294,4227.494042,58.945367,3474.556250,...,5.257409,0.060914,0.003711,0.123103,-1.382054,0.001643,5.186335,0.012475,5.208395,5.224619
2,1,4197,2014-06-12T04:24:00,2014-06-12T16:12:00,5971.995387,5427.601612,5685.931906,5676.449862,151.055841,22817.867146,...,5.310882,0.069291,0.004801,0.055277,-1.166496,0.001108,5.413816,0.009224,5.369353,5.349381
3,1,3686,2014-02-05T22:00:00,2014-02-06T09:48:00,8722.341628,7849.455657,8161.169937,8172.512223,234.807174,55134.408776,...,5.235223,0.082162,0.006751,0.449198,-1.156580,0.001324,5.396841,0.010436,5.304679,5.277076
4,1,3686,2014-02-05T15:00:00,2014-02-06T02:48:00,9240.027830,7849.455657,8572.241393,8476.056740,426.958915,182293.915002,...,5.228545,0.062445,0.003899,-0.256370,-1.382391,0.001280,5.166364,0.010703,5.180559,5.197977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42505,0,3320,2013-11-04T21:12:00,2013-11-05T09:00:00,67.561541,52.969193,59.883312,60.457297,3.201471,10.249419,...,0.059618,0.323763,0.104823,5.333765,27.360505,0.005126,1.790846,0.030353,0.171589,0.116305
42506,1,3415,2013-12-01T02:00:00,2013-12-01T13:48:00,789.630346,698.124698,748.062765,737.484348,26.081033,680.220286,...,1.920342,1.066029,1.136418,-1.103907,-0.381099,0.063195,0.000000,0.425484,1.324007,1.572680
42507,0,3705,2014-02-04T14:12:00,2014-02-05T02:00:00,11.522003,2.662429,5.113660,5.602774,2.237173,5.004944,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
42508,0,3996,2014-04-11T01:00:00,2014-04-11T12:48:00,1006.345520,844.060934,928.770780,931.624050,39.566040,1565.471495,...,4.876126,0.054204,0.002938,-0.483670,-0.286243,0.002584,4.782547,0.019712,4.833546,4.850094


In [4]:
df  = df.drop(['start_time','end_time'],axis=1)

In [5]:
Y = df.copy()['class']
X = df.copy().drop(['class','active_region'],axis=1)

GROUP = df.copy()['active_region']

In [6]:
def add_gaussian_noise(X_train,y_train,minority=1,samples=30000,sigma=0.01, random_state=42):

    X_train = pd.DataFrame(X_train).copy()
    y_train = pd.Series(y_train).copy()


    minority_mask = (y_train == minority) # query T F for row in target minority 
    X_minority = X_train[minority_mask].copy()

    Sampled_X = X_minority.sample(n=samples,replace=True,random_state=random_state)

    #ensure numerica columns here 


    X_num = Sampled_X.select_dtypes(include=[np.number])
    feature_std = X_num.std(axis=0).replace(0,1e-16)

    np.random.seed(random_state)
    noise = np.random.normal(loc=0,scale=sigma,size=X_num.shape) * feature_std.values

    X_noise_num = X_num + noise

    X_noise = Sampled_X.copy()
    X_noise[X_num.columns] = X_noise_num

    y_noise = pd.Series([minority] * samples,index=X_noise.index)

    X_aug = pd.concat([X_train,X_noise], ignore_index=True)
    Y_aug = pd.concat([y_train,y_noise.reset_index(drop=True)],ignore_index=True)

    return X_aug,Y_aug

In [7]:
Y = df.copy()['class']
X = df.copy().drop(['class','active_region'],axis=1)

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping

groups = GROUP

gkf = GroupKFold(n_splits=5)

oof_proba = pd.Series(index=Y.index, dtype=float)
fold_aucs = []

for fold, (tr_idx, va_idx) in enumerate(gkf.split(X, Y, groups=groups), 1):
    X_tr, X_va = X.iloc[tr_idx].copy(), X.iloc[va_idx].copy()
    y_tr, y_va = Y.iloc[tr_idx].copy(), Y.iloc[va_idx].copy()

    # noise ONLY on training fold
    X_tr, y_tr = add_gaussian_noise(X_tr, y_tr)

    # scale per fold (avoid leakage)
    scaler = StandardScaler()
    X_tr_scaled = pd.DataFrame(
        scaler.fit_transform(X_tr),
        columns=X_tr.columns,
        index=X_tr.index
    )
    X_va_scaled = pd.DataFrame(
        scaler.transform(X_va),
        columns=X_va.columns,
        index=X_va.index
    )

    # class imbalance weight from training fold
    neg = (y_tr == 0).sum()
    pos = (y_tr == 1).sum()
    spw = neg / max(pos, 1)

    model = XGBClassifier(
        n_estimators=5000,
        learning_rate=0.01,
        max_depth=6,
        objective="binary:logistic",
        eval_metric="logloss",
        scale_pos_weight=spw,
        seed =42,
        random_state=42,
        n_jobs=3,
    )

    model.fit(
        X_tr_scaled, y_tr,
        eval_set=[(X_va_scaled, y_va)],
        verbose=False
    )

    proba_va = model.predict_proba(X_va_scaled)[:, 1]
    oof_proba.loc[X_va_scaled.index] = proba_va

    auc = roc_auc_score(y_va, proba_va)
    fold_aucs.append(auc)

    print(f"Fold {fold} AUC: {auc:.4f}")

print(f"\nMean AUC: {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
print(f"OOF AUC:  {roc_auc_score(Y.loc[oof_proba.index], oof_proba):.4f}")

# threshold metrics
threshold = 0.5
oof_pred = (oof_proba >= threshold).astype(int)

print(confusion_matrix(Y.loc[oof_proba.index], oof_pred))
print(classification_report(Y.loc[oof_proba.index], oof_pred))


Fold 1 AUC: 0.9211
Fold 2 AUC: 0.9072
Fold 3 AUC: 0.9323
Fold 4 AUC: 0.9425
Fold 5 AUC: 0.9323

Mean AUC: 0.9271 ± 0.0120
OOF AUC:  0.9272
[[32590  2172]
 [ 2515  5233]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.93     34762
           1       0.71      0.68      0.69      7748

    accuracy                           0.89     42510
   macro avg       0.82      0.81      0.81     42510
weighted avg       0.89      0.89      0.89     42510



In [9]:
importances = model.feature_importances_
feature_names = X_tr_scaled.columns

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print(importance_df)

                               feature  importance
300                        R_VALUE_Min    0.246705
9                   TOTUSJH_last_value    0.036854
8    TOTUSJH_avg_abs_derivative_change    0.034477
92                           TOTFZ_Min    0.032741
70                    SAVNCPP_Variance    0.029301
..                                 ...         ...
20                     TOTBSQ_Kurtosis    0.000245
46                    TOTUSJZ_Kurtosis    0.000240
124                      EPSZ_Kurtosis    0.000238
280                      EPSY_Kurtosis    0.000235
163                   MEANGAM_Kurtosis    0.000185

[312 rows x 2 columns]


In [10]:
importance_df.to_csv('../webapp/XGBoost/XGFeatureImportance.csv')

In [11]:

top_22_features = pd.DataFrame({
    'feature': X_tr_scaled.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False).head(22)['feature'].tolist()

top_22_features

['R_VALUE_Min',
 'TOTUSJH_last_value',
 'TOTUSJH_avg_abs_derivative_change',
 'TOTFZ_Min',
 'SAVNCPP_Variance',
 'TOTUSJH_Max',
 'TOTFZ_average_absolute_change',
 'TOTPOT_Variance',
 'TOTFZ_last_value',
 'ABSNJZH_Min',
 'SAVNCPP_average_absolute_change',
 'ABSNJZH_average_absolute_change',
 'TOTUSJH_Variance',
 'TOTPOT_Std_dev',
 'R_VALUE_Variance',
 'MEANPOT_Mean',
 'USFLUX_last_value',
 'TOTFZ_Variance',
 'TOTBSQ_Variance',
 'ABSNJZH_Variance',
 'MEANGBZ_Max',
 'TOTUSJZ_last_value']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit

Y = df.copy()['class']
X = df.copy().drop(['class','active_region'],axis=1)

X = X[top_22_features]


GROUP = df.copy()['active_region']



X_train,X_test,y_train,y_test = train_test_split(X,Y,random_state=42,test_size=0.2,shuffle=True,stratify=Y)

scaler = StandardScaler()


# Use GroupShuffleSplit to split by active region
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, Y, groups=GROUP))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = Y.iloc[train_idx], Y.iloc[test_idx]

# check for  no overlap between ARS
train_regions = set(GROUP.iloc[train_idx])
test_regions = set(GROUP.iloc[test_idx])
print(f"Train regions: {len(train_regions)}")
print(f"Test regions: {len(test_regions)}")
print(f"Overlap: {len(train_regions & test_regions)}")  # Should be 0! if not then leakage 

#scaling is needed for XGBoost in this scenario. 
#theres datapoints that are too massive and throws an error so we need to scale and then add gausasin noise
scaler = StandardScaler()
X_train, y_train = add_gaussian_noise(X_train, y_train,samples=5000)
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# class imbalance weight from training fold
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
spw = neg / max(pos, 1)

model = XGBClassifier(
    n_estimators=5000,
    learning_rate=0.01,
    max_depth=6,
    objective="binary:logistic",
    eval_metric="logloss",
    scale_pos_weight=spw,
    seed=42,
    random_state=42,
    n_jobs=3,
)

model.fit(
    X_train_scaled, y_train,
    eval_set=[(X_test_scaled, y_test)],
    verbose=False
)


from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# ROC AUC score
print(f"\nROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")





# Evaluate on held-out test set
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print(f"\nROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

# model deplotmnet
import pickle

# Save the scaler and model
with open('../models/XGBoost/scaler_v2.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
model.save_model('../models/XGBoost/XGBoostModel_v2.ubj')

# Also save feature names for validation
with open('../models/XGBoost/feature_names_v2.pkl', 'wb') as f:
    pickle.dump(X_train.columns.tolist(), f)

print("\n Model and scaler saved!")

Train regions: 312
Test regions: 78
Overlap: 0
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      6754
           1       0.61      0.60      0.61      1405

    accuracy                           0.87      8159
   macro avg       0.77      0.76      0.76      8159
weighted avg       0.86      0.87      0.87      8159


Confusion Matrix:
[[6224  530]
 [ 562  843]]

ROC AUC Score: 0.9058
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      6754
           1       0.61      0.60      0.61      1405

    accuracy                           0.87      8159
   macro avg       0.77      0.76      0.76      8159
weighted avg       0.86      0.87      0.87      8159


Confusion Matrix:
[[6224  530]
 [ 562  843]]

ROC AUC Score: 0.9058

 Model and scaler saved!
